In [2]:
import pandas as pd
data = pd.read_csv("train.csv")

In [3]:
data.head()

,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
0,1,b,18.42,10.415,y,p,aa,v,0.125,t,f,0,f,g,120.0,375,1
1,2,a,21.75,11.750,u,g,c,v,0.250,f,f,0,t,g,180.0,0,1
2,3,b,30.17,1.085,y,p,c,v,0.040,f,f,0,f,g,170.0,179,1
3,4,b,22.67,2.540,y,p,c,h,2.585,t,f,0,f,g,0.0,0,0
4,5,a,36.00,1.000,u,g,c,v,2.000,t,t,11,f,g,0.0,456,0


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score,precision_recall_curve,f1_score,accuracy_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

data_train = pd.read_csv("train.csv")

###Replace Missing Values with median for catagorical Columns


data_train.B.fillna(28.17,inplace = True)
data_train.N.fillna(152,inplace = True)

#Treating Missing Value as a neew_catagory for catagorial columns

data_train.fillna('NN',inplace = True)

### saving the target varaible

target = data_train.P

### Columns D is same as E so we can drop any of them. Here dropping D and Keeping E.
### Column J is irrevent for this classification problem. So drop J.
data_train.drop(['D','J','P','id'],axis =1,inplace =True)


data_train_int = data_train.select_dtypes(include=['float64','int64']).copy()

data_train_cat = data_train.select_dtypes(include=['object']).copy()

train_colums = data_train_cat.columns
label_dict ={}

for each in train_colums:
    la = LabelEncoder().fit(data_train_cat[each])
    data_train_cat[each] = la.transform(data_train_cat[each])
   
    label_dict[each] = la
data_train_cat = data_train_cat.as_matrix()


one_he = OneHotEncoder()


data_train_X =one_he.fit_transform(data_train_cat)
data_train_X =data_train_X.toarray()
data_train_int = data_train_int.as_matrix()

#X_train = np.concatenate((data_train_int[:,1::],data_train_X ), axis=1)
#Y_train = target.as_matrix()
X = np.concatenate((data_train_int,data_train_X,np.sin(data_train_int),np.cos(data_train_int) ), axis=1)
Y = target.as_matrix()

from sklearn.model_selection import train_test_split
seed = 10
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=seed)
#X_train = X_train_al[:,1:data_set_one_hot.shape[1]]
#X_test = X_test_al[:,1:data_set_one_hot.shape[1]]



#xgb_model = XGBClassifier(booster='gbtree',max_depth=4,n_estimators=1000,learning_rate=0.0555)
#xgb_model.fit(X_train, y_train)

model = XGBClassifier(booster='gbtree',max_depth=4,n_estimators=900,learning_rate=0.055)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

from sklearn import svm
from sklearn import preprocessing
leh = preprocessing.StandardScaler()
leh.fit(X_train)
X_train_sc  =leh.transform(X_train)
X_test_sc =leh.transform(X_test)
clf_svm = svm.SVC(C=1,gamma=.001)
clf_svm.fit(X_train_sc, y_train)  

pred_s = clf_svm.predict(X_test_sc)
accuracy = accuracy_score(y_test, pred_s)
print("Accuracy_svm: %.2f%%" % (accuracy * 100.0))

# data_test = pd.read_csv("test.csv")

# data_test.drop(['D','J'],axis =1,inplace =True)
# data_test.B.fillna(28.17,inplace = True)
# data_test.N.fillna(152,inplace = True)


# data_test.fillna('NN',inplace = True)





# data_test_int = data_test.select_dtypes(include=['float64','int64']).copy()

# data_test_cat = data_test.select_dtypes(include=['object']).copy()

# test_colums = data_test_cat.columns

# for each in test_colums:
#     la = label_dict[each]
#     data_test_cat[each] = la.transform(data_test_cat[each])
   
   

# data_test_cat = data_test_cat.as_matrix()



# data_test_X =one_he.transform(data_test_cat)
# data_test_X =data_test_X.toarray()
# data_test_int = data_test_int.as_matrix()
# X_test = np.concatenate((data_test_int[:,1::],data_test_X ), axis=1)






# predicted_class = xgb_model.predict(X_test)


# result = pd.DataFrame({'id': data_test_int[:,0],'P': predicted_class})
# result.to_csv('submission.csv',index = False)

from sklearn.ensemble import RandomForestClassifier

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_estimators=350,max_depth=4,criterion ='entropy', random_state=100)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(X_train, y_train)

# Apply the Classifier we trained to the test data (which, remember, it has never seen before)
pre = clf.predict(X_test)

#clf.predict_proba(test[features])[0:10]

accuracy = accuracy_score(y_test, pre)
print("random Accuracy: %.2f%%" % (accuracy * 100.0))

/home/hbbg/shyam_works/vir_env/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/hbbg/shyam_works/vir_env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/hbbg/shyam_works/vir_env/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/hbbg/shyam_works/vir_env/lib/python3.6/sit

Accuracy: 86.96%
Accuracy_svm: 85.51%
random Accuracy: 86.96%


In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[58,  5],
       [ 9, 66]])